In [1]:
# -------------------------
# 📘 Nhận diện khuôn mặt bằng Haar Cascade
# Người thực hiện: Ngô Quang Vũ - 197CT10170
# ------------------------

import cv2
import os
import datetime

# -----------------------------------------------
# Load Haar Cascade model
# -----------------------------------------------
haar_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
face_cascade = cv2.CascadeClassifier(haar_path)

# -----------------------------------------------
# Hàm xử lý nhận diện khuôn mặt
# -----------------------------------------------
def detect_faces(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )
    return faces

# -----------------------------------------------
# 1. Nhận diện từ ảnh
# -----------------------------------------------
def detect_from_image(image_path):
    if not os.path.exists(image_path):
        print(" Không tìm thấy ảnh:", image_path)
        return

    image = cv2.imread(image_path)
    faces = detect_faces(image)

    print(f"✔️ Phát hiện {len(faces)} khuôn mặt trong ảnh.")

    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Hiển thị ảnh
    cv2.imshow("Ket qua nhan dien tu anh", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Lưu ảnh vào folder result/
    os.makedirs("result", exist_ok=True)
    filename = os.path.basename(image_path)
    output_path = os.path.join("result", f"detected_{filename}")
    cv2.imwrite(output_path, image)
    print(f"💾 Ảnh đã được lưu vào: {output_path}")


# -----------------------------------------------
# 2. Nhận diện từ video
# -----------------------------------------------
def detect_from_video(video_path):
    if not os.path.exists(video_path):
        print(" Không tìm thấy video:", video_path)
        return

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(" Không thể mở video.")
        return

    os.makedirs("result", exist_ok=True)
    frame_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_id += 1
        faces = detect_faces(frame)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        cv2.imshow("Nhan dien tu video", frame)

        # Lưu frame có khuôn mặt vào thư mục result
        if len(faces) > 0:
            filename = f"result/Haar_Cascade{frame_id}.jpg"
            cv2.imwrite(filename, frame)
            print(f"💾 Đã lưu khung có khuôn mặt vào: {filename}")

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


# -----------------------------------------------
# 3. Nhận diện từ webcam (quét liên tục đến khi nhấn 'e' hoặc đóng)
# -----------------------------------------------
def detect_from_webcam():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print(" Không thể mở webcam.")
        return

    print("🎥 Đang dùng webcam. Đang tìm khuôn mặt...")

    # Tạo thư mục resuilt_haar_cascade nếu chưa có
    os.makedirs("resuilt_haar_cascade", exist_ok=True)
    log_file = open("log.txt", "w", encoding="utf-8")

    frame_id = 0
    frame_count_with_faces = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_id += 1
        faces = detect_faces(frame)

        if len(faces) > 0:
            frame_count_with_faces += 1

            print(f"[{frame_id}] ✔️ Phát hiện {len(faces)} khuôn mặt. Đã lưu ảnh.")

            # Ghi log
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            log_file.write(f"[{timestamp}] Frame {frame_id}: {len(faces)} faces detected.\n")

            # Vẽ khung lên ảnh
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                cv2.putText(frame, "Face detected", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Lưu ảnh vào thư mục mới
            image_path = f"resuilt_haar_cascade/Haar_Cascade_{frame_id}.jpg"
            cv2.imwrite(image_path, frame)

        # Hiển thị ảnh
        cv2.imshow("Phat hien khuon mat", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('e') or key == 27:  # Nhấn 'e' hoặc ESC để thoát
            print(" Đã thoát quét webcam.")
            break

    cap.release()
    cv2.destroyAllWindows()
    log_file.close()

    print(f" Tổng số khung có khuôn mặt được lưu: {frame_count_with_faces}")




# -----------------------------------------------
# Menu chính
# -----------------------------------------------
# Menu chính
def main():
    while True:
        print("\n=== Chương trình Nhận diện Khuôn mặt bằng Haar Cascade ===")
        print("1. Nhận diện từ Webcam")
        print("2. Nhận diện từ Ảnh")
        print("3. Nhận diện từ Video")
        print("4. Thoát chương trình")
        choice = input(" Nhập lựa chọn (1/2/3/4): ")

        if choice == '1':
            detect_from_webcam()
        elif choice == '2':
            path = input(" Nhập đường dẫn tới ảnh: ")
            detect_from_image(path)
        elif choice == '3':
            path = input(" Nhập đường dẫn tới video: ")
            detect_from_video(path)
        elif choice == '4':
            print(" Đã thoát chương trình.")
            break
        else:
            print(" Lựa chọn không hợp lệ.")


if __name__ == "__main__":
    main()



=== Chương trình Nhận diện Khuôn mặt bằng Haar Cascade ===
1. Nhận diện từ Webcam
2. Nhận diện từ Ảnh
3. Nhận diện từ Video
4. Thoát chương trình
 Đã thoát chương trình.
